In [ ]:
!pip install pennylane
!pip install tensorflow-hub
!pip install tensorflow-datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 39.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pennylane as qml
import torch
import numpy as np
import random
import torch.nn as nn
from tqdm import tqdm

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.12.0
Eager mode:  True
Hub version:  0.13.0
GPU is NOT AVAILABLE


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
path='/content/gdrive/MyDrive/New_test/'

In [ ]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteXK588J/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteXK588J/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteXK588J/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(15000)))
train_examples_batch

<tf.Tensor: shape=(15000,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fe

In [ ]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

In [ ]:
ten=hub_layer(train_examples_batch)
Zs=np.zeros((15000,2),dtype=np.float32)
X_train=torch.tensor(np.concatenate((ten.numpy(),Zs),axis=1))
Y_train=torch.tensor(train_labels_batch.numpy())

In [ ]:
New_X_train=X_train[:2000]
new_Y_train=Y_train[:2000]

In [ ]:
new_X_train=[]
for seq in New_X_train:
    
    new_X_train.append(torch.reshape(seq, (13, 4)))

new_X_train=torch.stack(new_X_train)

In [ ]:
import functools
import inspect
import math
from collections.abc import Iterable
from typing import Callable, Dict, Union, Any

from pennylane.qnode import QNode

try:
    import torch
    from torch.nn import Module

    TORCH_IMPORTED = True
except ImportError:
    # The following allows this module to be imported even if PyTorch is not installed. Users
    # will instead see an ImportError when instantiating the TorchLayer.
    from unittest.mock import Mock

    Module = Mock
    TORCH_IMPORTED = False


class TorchLayer(Module):
    def __init__(self,qnode,weights):
        if not TORCH_IMPORTED:
            raise ImportError(
                "TorchLayer requires PyTorch. PyTorch can be installed using:\n"
                "pip install torch\nAlternatively, "
                "visit https://pytorch.org/get-started/locally/ for detailed "
                "instructions."
            )
        super().__init__()

        #weight_shapes = {
        #    weight: (tuple(size) if isinstance(size, Iterable) else () if size == 1 else (size,))
        #    for weight, size in weight_shapes.items()
        #}

        # validate the QNode signature, and convert to a Torch QNode.
        # TODO: update the docstring regarding changes to restrictions when tape mode is default.
        #self._signature_validation(qnode, weight_shapes)
        self.qnode = qnode
        self.qnode.interface = "torch"

        self.qnode_weights = weights

    def forward(self, inputs):  # pylint: disable=arguments-differ
        """Evaluates a forward pass through the QNode based upon input data and the initialized
        weights.

        Args:
            inputs (tensor): data to be processed

        Returns:
            tensor: output data
        """

        if len(inputs.shape) > 1:
            # If the input size is not 1-dimensional, unstack the input along its first dimension,
            # recursively call the forward pass on each of the yielded tensors, and then stack the
            # outputs back into the correct shape
            reconstructor = [self.forward(x) for x in torch.unbind(inputs)]
            return torch.stack(reconstructor)

        # If the input is 1-dimensional, calculate the forward pass as usual
        return self._evaluate_qnode(inputs)


    def _evaluate_qnode(self, x):
        """Evaluates the QNode for a single input datapoint.

        Args:
            x (tensor): the datapoint

        Returns:
            tensor: output datapoint
        """
        kwargs = {
            **{self.input_arg: x},
            **{arg: weight.to(x) for arg, weight in self.qnode_weights.items()},
        }
        res = self.qnode(**kwargs)

        if isinstance(res, torch.Tensor):
            return res.type(x.dtype)

        return torch.hstack(res).type(x.dtype)

    def __str__(self):
        detail = "<Quantum Torch Layer: func={}>"
        return detail.format(self.qnode.func.__name__)

    __repr__ = __str__
    _input_arg = "inputs"

    @property
    def input_arg(self):
        """Name of the argument to be used as the input to the Torch layer. Set to ``"inputs"``."""
        return self._input_arg

In [ ]:


class QSAL_pennylane(torch.nn.Module):
    def __init__(self,S,n,D):
        super().__init__()
        self.seq_num=S
        self.init_params_Q=torch.nn.Parameter(torch.stack([(np.pi/4) * (2 * torch.randn(n*(D+2)) - 1) for _ in range(self.seq_num)]))
        self.init_params_K=torch.nn.Parameter(torch.stack([(np.pi/4) * (2 * torch.randn(n*(D+2)) - 1) for _ in range(self.seq_num)]))
        self.init_params_V=torch.nn.Parameter(torch.stack([(np.pi/4) * (2 * torch.randn(n*(D+2)) - 1) for _ in range(self.seq_num)]))
        self.num_q=n
        #self.Denc=Denc
        self.D=D
        self.d=2**n#(Denc+2)
        self.dev = qml.device("default.qubit", wires=self.num_q)
        
        
        self.vqnod=qml.QNode(self.circuit_v, self.dev, interface="torch")
        self.qnod=qml.QNode(self.circuit_qk, self.dev, interface="torch")
        self.weight_v = [{"weights": self.init_params_V[i]} for i in range(self.seq_num)]
        self.weight_q = [{"weights": self.init_params_Q[i]} for i in range(self.seq_num)]
        self.weight_k = [{"weights": self.init_params_K[i]} for i in range(self.seq_num)]
        #self.v_linear ={} #[qml.qnn.TorchLayer(self.vqnod[i], self.weight_shapes) for i in range(self.seq_num)]
        #for i in range(self.seq_num):
        self.v_linear = [TorchLayer(self.vqnod, self.weight_v[i]) for i in range(self.seq_num)]
        self.q_linear = [TorchLayer(self.qnod, self.weight_q[i]) for i in range(self.seq_num)]
        self.k_linear = [TorchLayer(self.qnod, self.weight_k[i]) for i in range(self.seq_num)]
        #self.qqnod=[qml.QNode(self.circuit_qk, self.dev, interface="torch") for i in range(self.seq_num)]

    def random_op(self):
        a=random.randint(0, 4)
        if a==0:
            op=qml.Identity(0)
        elif a==1:
            op=qml.PauliX(0)
        elif a==2:
            op=qml.PauliY(0)
        else:
            op=qml.PauliZ(0)

        op_elimated=qml.Identity(0)
        for i in range(1,self.num_q):
            op_elimated=op_elimated@qml.Identity(i)
        Select_wrong=True
        while Select_wrong:
            for i in range(1,self.num_q):
                a=random.randint(0, 4)
                if a==0:
                    op=op@qml.Identity(i)
                elif a==1:
                    op=op@qml.PauliX(i)
                elif a==2:
                    op=op@qml.PauliY(i)
                else:
                    op=op@qml.PauliZ(i)
            if op!=op_elimated:
                Select_wrong=False
        return op

    def circuit_v(self,inputs,weights):
        op=self.random_op()
        qml.AmplitudeEmbedding(inputs,wires=range(self.num_q), normalize=True)
        # Ansatz
        indx=0
        for j in range(self.num_q):
            qml.RX(weights[indx],j)
            qml.RY(weights[indx+1],j)
            indx+=2
        for i in range(self.D):
            for j in range(self.num_q):
                qml.CNOT(wires=(j,(j+1)%self.num_q))
                
            for j in range(self.num_q):
                #qc.rx(params[indx],j)
                qml.RY(weights[indx],j)
                indx+=1
        return [qml.expval(op) for i in range(self.d)] 

    def circuit_qk(self,inputs,weights):
        op=self.random_op()
        # feature_map
        qml.AmplitudeEmbedding(inputs,wires=range(self.num_q), normalize=True)
        # Ansatz
        indx=0
        for j in range(self.num_q):
            qml.RX(weights[indx],j)
            qml.RY(weights[indx+1],j)
            indx+=2
        for i in range(self.D):
            for j in range(self.num_q):
                qml.CNOT(wires=(j,(j+1)%self.num_q))
                
            for j in range(self.num_q):
                #qc.rx(params[indx],j)
                qml.RY(weights[indx],j)
                indx+=1
        return [qml.expval(qml.PauliZ(0))]

    def forward(self,input):

        Q_output=torch.stack([self.q_linear[i](input[:,i]) for i in range(self.seq_num)])
        K_output=torch.stack([self.k_linear[i](input[:,i]) for i in range(self.seq_num)])
        V_output=torch.stack([self.v_linear[i](input[:,i]) for i in range(self.seq_num)])
        
        batch_size=len(input)
        Q_output=Q_output.transpose(0,2).repeat((self.seq_num,1,1))
        K_output=K_output.transpose(0,2).repeat((self.seq_num,1,1)).transpose(0,2)
        #print(V_output.size())
        #Q_grid, K_grid=torch.meshgrid(Q_output, K_output, indexing='ij')
        alpha=torch.exp(-(Q_output-K_output)**2)
        alpha=alpha.transpose(0,1)
        V_output=V_output.transpose(0,1)
        output=[]

        for i in range(self.seq_num):
            
            Sum_a=torch.sum(alpha[:,i,:],-1)
            div_sum_a=(1/Sum_a).repeat(self.d,self.seq_num,1).transpose(0,2)
            
            Sum_w=torch.sum(alpha[:,:,i].repeat((self.d,1,1)).transpose(0,2).transpose(0,1)*V_output*div_sum_a,1)
            output.append(Sum_w)
        return input+torch.stack(output).transpose(0,1)

class QSANN_pennylane(torch.nn.Module):
    def __init__(self,S,n,D,num_layers):
        """
        # input: input data
        # weight: trainable parameter
        # n: # of of qubits
        # d: embedding dimension which is equal to n(Denc+2)
        # Denc: the # number of layers for encoding 
        # D: the # of layers of variational layers
        # type "K": key, "Q": Query, "V": value
        """
        super().__init__()
        self.qsal_lst=[QSAL_pennylane(S,n,D) for _ in range(num_layers)]
        self.qnn=nn.Sequential(*self.qsal_lst)

    def forward(self,input):

        return self.qnn(input)

class QSANN_text_classifier(torch.nn.Module):
    def __init__(self,S,n,D,num_layers):
        """
        # input: input data
        # weight: trainable parameter
        # n: # of of qubits
        # d: embedding dimension which is equal to n(Denc+2)
        # Denc: the # number of layers for encoding 
        # D: the # of layers of variational layers
        # type "K": key, "Q": Query, "V": value
        """
        super().__init__()
        self.Qnn=QSANN_pennylane(S,n,D,num_layers)
        self.final_layer=nn.Linear((2**n)*S, 1)
        self.final_layer=self.final_layer.float()

    def forward(self,input):

        x=self.Qnn(input)
        x=torch.flatten(x,start_dim=1)
        
        return torch.sigmoid(self.final_layer(x))

# Toy data

In [ ]:
X=[[np.array([random.random() for _ in range(8)]) for _ in range(5)] for _ in range(32)]
Y=[random.randint(0,1) for _ in range(32)]

In [ ]:
model=QSANN_text_classifier(5,3,1,2)

In [ ]:
optimizer = torch.optim.Adam(lr=0.03, params=model.parameters())

In [ ]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(trainable_params)

311


In [ ]:
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = (torch.round(torch.sign(preds-0.5))+1)//2
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
for iepoch in tqdm(range(10)):
    optimizer.zero_grad()
    X_tensor=torch.tensor(X)
    predictions=model(X_tensor.float()).squeeze(1)
    #predictions=torch.sign(predictions)
    label=torch.tensor(Y)
    loss = criterion(predictions, label.float())
    acc = binary_accuracy(predictions, label)
    print('')
    print('Accuracy:',acc)
    print('')
    print(loss)
    loss.backward()
    optimizer.step()

  0%|          | 0/10 [00:00<?, ?it/s]


Accuracy: tensor(0.3438)

tensor(62.5064, grad_fn=<DivBackward1>)


 10%|█         | 1/10 [00:15<02:21, 15.71s/it]


Accuracy: tensor(0.6562)

tensor(62.1799, grad_fn=<DivBackward1>)


 20%|██        | 2/10 [00:31<02:04, 15.60s/it]

...

 80%|████████  | 8/10 [02:07<00:31, 15.86s/it]


Accuracy: tensor(0.5312)

tensor(62.3118, grad_fn=<DivBackward1>)


 90%|█████████ | 9/10 [02:23<00:16, 16.16s/it]


Accuracy: tensor(0.5938)

tensor(62.1264, grad_fn=<DivBackward1>)


100%|██████████| 10/10 [02:39<00:00, 15.99s/it]


# Real Data

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = (torch.round(torch.sign(preds-0.5))+1)//2
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
model = QSANN_text_classifier(13,2,1,1)

In [ ]:
optimizer = torch.optim.Adam(lr=0.005, params=model.parameters())
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(trainable_params)

287


In [ ]:
label=new_Y_train

In [ ]:
for iepoch in tqdm(range(50)):
    
    optimizer.zero_grad()

    idx = torch.randperm(2000)[:400]
    X_batch = new_X_train[idx]
    predictions=model(X_batch.float()).squeeze(1)

    label_batch=label[idx]
    loss = criterion(predictions, label_batch.float())
    acc = binary_accuracy(predictions,label_batch)
    print('')
    print('Accuracy:',acc)
    print('')
    print(loss)
    loss.backward()
    optimizer.step()

  0%|          | 0/50 [00:00<?, ?it/s]


Accuracy: tensor(0.4475)

tensor(1211.3503, grad_fn=<DivBackward1>)


  2%|▏         | 1/50 [02:31<2:03:38, 151.40s/it]


Accuracy: tensor(0.4750)

tensor(1264.9314, grad_fn=<DivBackward1>)


  4%|▍         | 2/50 [05:07<2:03:19, 154.15s/it]

...

 96%|█████████▌| 48/50 [2:11:09<05:33, 166.97s/it]


Accuracy: tensor(0.6675)

tensor(1184.2261, grad_fn=<DivBackward1>)


 98%|█████████▊| 49/50 [2:13:54<02:46, 166.23s/it]


Accuracy: tensor(0.6750)

tensor(1207.5037, grad_fn=<DivBackward1>)


100%|██████████| 50/50 [2:16:41<00:00, 164.02s/it]


In [ ]:
X_batch = new_X_train
predictions=model(X_batch.float()).squeeze(1)

loss = criterion(predictions, label.float())
acc = binary_accuracy(predictions,label)
print('')
print('Accuracy:',acc)
print('')
print(loss)

In [ ]:
label_test=torch.stack(new_y_test)

In [ ]:
X_batch = torch.stack(new_X_test)
predictions=model(X_batch.float()).squeeze(1)

loss = criterion(predictions, label_test.float())
acc = binary_accuracy(predictions,label_test)
print('')
print('Accuracy:',acc)
print('')
print(loss)

In [ ]:
file_name=path+'QSANN_pennylane_model_1'
torch.save(model.state_dict(),file_name)

In [ ]:
for iepoch in tqdm(range(50)):
    
    optimizer.zero_grad()

    idx = torch.randperm(2000)[:400]
    X_batch = new_X_train[idx]
    predictions=model(X_batch.float()).squeeze(1)

    label_batch=label[idx]
    loss = criterion(predictions, label_batch.float())
    acc = binary_accuracy(predictions,label_batch)
    print('')
    print('Accuracy:',acc)
    print('')
    print(loss)
    loss.backward()
    optimizer.step()

  0%|          | 0/50 [00:00<?, ?it/s]


Accuracy: tensor(0.6900)

tensor(1148.9222, grad_fn=<DivBackward1>)


  2%|▏         | 1/50 [02:45<2:15:02, 165.36s/it]


Accuracy: tensor(0.6725)

tensor(1216.3080, grad_fn=<DivBackward1>)


  4%|▍         | 2/50 [05:32<2:13:06, 166.38s/it]

...

 96%|█████████▌| 48/50 [2:12:01<05:27, 163.92s/it]


Accuracy: tensor(0.7325)

tensor(1137.4692, grad_fn=<DivBackward1>)


 98%|█████████▊| 49/50 [2:14:46<02:44, 164.18s/it]


Accuracy: tensor(0.6900)

tensor(1226.2415, grad_fn=<DivBackward1>)


100%|██████████| 50/50 [2:17:34<00:00, 165.09s/it]
